In [27]:
cd "h:\내 드라이브\Inventory_Order_update"

h:\내 드라이브\Inventory_Order_update


In [28]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pandas as pd
import json

root_path=''

In [29]:
with open(root_path+'appdata/db_auth.json') as f:
        temp = json.load(f)
        server = temp['server']
        database = temp['database'] 
        username = temp['username'] 
        password = temp['password']

connection_string = 'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = create_engine(connection_url)

In [30]:
query = f"""
SELECT 
    item.ItemLookupCode, 
    item.Quantity,
    sp.SupplierID, 
    dbo.Supplier.SupplierName, 
    dbo.Supplier.Code, 
    item.Description as Description,
    item.Price Price, 
    item.Cost itemCost, 
    sp.Cost spCost
FROM 
    dbo.Item item
LEFT JOIN 
    dbo.SupplierList sp ON item.ID=sp.ItemID
LEFT JOIN 
    dbo.Supplier ON sp.SupplierID=dbo.Supplier.ID
WHERE 
    item.DepartmentID IN (2,4,6)
    AND Item.Inactive = 0
ORDER BY 
    item.SupplierID,
    item.Description;
"""

with engine.connect() as conn, conn.begin():  
    fromPOS = pd.read_sql(query, conn, dtype={'ItemLookupCode': str})

In [31]:
all_amazon = pd.read_csv(root_path+'inv_data\Amazon_All+Listings+Report.txt', sep='\t')

all_amazon = all_amazon[['seller-sku', 'asin1', 'item-name', 'item-description',
'price', 'quantity', 'status', 'product-id']]

In [32]:
all_amazon

,seller-sku,asin1,item-name,item-description,price,quantity,status,product-id
0,AL_10 CLOSURE 4X4_44,B07SQG9Y6H,"Foxy Silver (Foxy Closure - 10"" Hh Lace) - Unp...",<b>Alicia Beauty Foxy Silver Collections Brazi...,58.9,10,Active,660014727678
1,AL_10 CLOSURE 4X4_51,B07SXQYHFJ,"Foxy Silver (Foxy Closure - 10"" HH Lace) - Unp...",<b>Alicia Beauty Foxy Silver Collections Brazi...,58.9,10,Active,660014727685
2,AL_AMARA_2,B0849V3KW2,Foxy Lady (Amara - Synthetic Full Wig in 2,<b>Foxy Lady Beautiful Young Straight Short Le...,23.9,10,Active,660014731323
3,AL_AMARA_4,B0849YWDX8,Foxy Lady (Amara - Synthetic Full Wig in 4,<b>Foxy Lady Beautiful Young Straight Short Le...,23.9,10,Active,660014731330
4,AL_AMARA_613,B0849RD93L,Foxy Lady (Amara) - Synthetic Full Wig in 613,<b>Foxy Lady Beautiful Young Straight Short Le...,23.9,10,Active,660014731361
...,...,...,...,...,...,...,...,...
43028,VF_YESICA_P,B086QNCSHN,"Vivica A. Fox High Heat Resistant Wigs, 26 Inc...",NaN,NaN,0,Incomplete,B086QNCSHN
43029,VF_YUCERA_P,B01B808WSC,Vivica A Fox Yucera Synthetic Fiber Natural Ba...,NaN,NaN,0,Incomplete,B01B808WSC
43030,VF_YUKI_P,B0C8ZQRNDS,Vivica A. Fox OXFORD - Remi Natural Human Hair...,NaN,NaN,0,Incomplete,B0C8ZQRNDS
43031,VF_ZINNIA_P,B09NBHCQDZ,VIVICA A FOX NATURAL HUMAN HAIR WIG - ZINNIA,NaN,NaN,0,Incomplete,B09NBHCQDZ


In [33]:
temp = all_amazon.merge(fromPOS[['ItemLookupCode', 'Price', 'itemCost', 'spCost']], how='left', left_on='product-id', right_on='ItemLookupCode')

In [34]:
temp2 = temp.fillna('')

In [35]:
temp3 = temp2[temp2.duplicated()]

In [36]:
check_list = temp[temp['price']<temp['Price']]

In [37]:
temp.drop('ItemLookupCode', axis=1, inplace=True)

In [38]:
temp['Cost'] = temp[['itemCost', 'spCost']].max(axis=1)
temp.drop(['itemCost', 'spCost'], axis=1, inplace=True)
temp['Cost'].fillna(0, inplace=True)

In [ ]:
temp['spCost']

In [11]:
temp.to_excel('amazon_pirce_check.xlsx', index=False)

In [18]:
temp[['itemCost', 'spCost']].max(axis=1)

0        49.95
1        49.95
2        10.95
3        10.95
4        10.95
         ...  
43085      NaN
43086      NaN
43087      NaN
43088      NaN
43089      NaN
Length: 43090, dtype: float64

In [41]:
brs_inv = pd.read_excel('inv_data/AL_brs inv.xls')
inv = pd.read_excel('inv_data/AL_inv.xls')

In [42]:
al_inv = pd.concat([brs_inv, inv], ignore_index=True)
al_inv.drop('Stock Lvl', inplace=True)
al_inv.columns = ['ItemCode', 'ItemCodeDesc', 'Stock Lvl', 'O/H', 'UPC', 'spCost']

Index(['ItemCode', 'ItemCodeDesc', 'Stock Lvl', 'OnHand Customer',
       'AliasItemNo', 'StandardUnitPrice'],
      dtype='object')

: 

In [44]:
by_inv = pd.read_excel('inv_data/BY_InventoryListAll.xls', skiprows=3)

WARNING *** file size (5275520) not 512 + multiple of sector size (512)


In [52]:
by_inv.columns

Index(['Item Name', 'Unnamed: 1', 'Color', 'Class', 'O/H', 'Unnamed: 5',
       'Price', 'Barcode'],
      dtype='object')

In [54]:
by_inv.drop(columns=['Unnamed: 1','Unnamed: 5'], inplace=True)

In [58]:
by_inv[['Item Name', 'Color', 'Class', 'O/H', 'Barcode', 'Price']]

,Item Name,Color,Class,O/H,Barcode,Price
0,134 CANNA,1,LC,112,885160294489,26.50
1,134 CANNA,1B,LC,93,885160294496,26.50
2,134 CANNA,4N274,LC,22,885160294533,26.50
3,134 CANNA,BT4007,LC,22,885160294540,26.50
4,134 CANNA,F2842,LC,47,885160294502,26.50
...,...,...,...,...,...,...
31785,ZZR LOA,SNOW,LC,0,885160246532,20.95
31786,ZZR LOA,SP1B/30,LC,0,885160246464,20.95
31787,ZZR LOA,SP427,LC,0,885160252335,20.95
31788,ZZR LOA,SP430,LC,0,885160257224,20.95
